# RasterizePoints
Author: Nathan Wisla

Date: April 12, 2021

Purpose: to take points from a PostGIS database and to interpolate them as rasters


## Initializing the Workspace

In [17]:
import datetime
import arcpy

def APinitialize():
    ws = r'S:\GIS\ArcGIS\Private_Projects\Aurora\scratch.gdb'
    sr = arcpy.SpatialReference(4326)
    arcpy.env.workspace = ws
    arcpy.env.overwriteOutput = True
    return ws, sr

ws, sr = APinitialize()

ws2 = 'S:\\GIS\\ArcGIS\\Private_Projects\\Aurora\\aurora.sde'
auroraFC = 'aurora.public.aurorastrength'
aurora = arcpy.Describe(ws2 + '\\' + auroraFC)



## Creating 48 Spline Rasters based on database data

In [ ]:
for i in range(48):

            
        
    arcpy.management.SelectLayerByAttribute(aurora.name,\
                                      'NEW_SELECTION',\
                                      f"id = {i + 1}")

    out_raster = arcpy.sa.Spline(aurora.name, "strength", 0.1, "REGULARIZED", 0.1, 12)
    rasterList.append(out_raster)
    out_raster.save(ws + '\\' + f'spline{i+1}')

In [21]:
mosaic = 'RasterAnimated'
mosaic = arcpy.management.CreateMosaicDataset(ws,mosaic, sr, None, '8_BIT_UNSIGNED', "NONE", None)
mosaic = arcpy.Describe(mosaic)
arcpy.management.AddRastersToMosaicDataset(mosaic.name, "Raster Dataset", arcpy.ListRasters(), "UPDATE_CELL_SIZES", "UPDATE_BOUNDARY", "NO_OVERVIEWS", None, 0, 1500, None, '', "SUBFOLDERS", "ALLOW_DUPLICATES", "NO_PYRAMIDS", "NO_STATISTICS", "NO_THUMBNAILS", '', "NO_FORCE_SPATIAL_REFERENCE", "NO_STATISTICS", None, "NO_PIXEL_CACHE", r"C:\Users\Nathan\AppData\Local\ESRI\rasterproxies\raw")

mosaicTable = mosaic.name + '\\Footprint'



            

After finalizing the raster mosaic, animate the raster by manually adding the time field by adding a join from the point data attributes based on: 

mosaic.oid = pointdata.id (one-to-one)

From there, calculate a new datetime field.

In [22]:
arcpy.management.AddField(mosaicTable, 'forecast', 'DATE')
arcpy.management.AddJoin(mosaicTable, 'OBJECTID', auroraFC, "id", "KEEP_ALL")
arcpy.management.CalculateField(mosaicTable, 'forecast', '!aurora.public.%aurorastrength.forecast!', 'PYTHON3', '', 'TEXT')
arcpy.management.RemoveJoin(mosaicTable, "aurora.public.%aurorastrength")

<Result 'RasterAnimated\\Footprint'>